# Fashion MNIST with CNN


In [1]:
import tensorflow as tf

In [18]:
class on_accuracy_stop(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>=0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = [on_accuracy_stop()]

In [23]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = fashion_mnist.load_data()

#Preprocessing data
training_images = tf.expand_dims(training_images, -1) # Conv2D uses a 4D array, increase the dim appending a value
test_images = tf.expand_dims(test_images, -1)

In [27]:
#optimize data
# 1. Turn train and test arrays into tensor Datasets
train_features_dataset = tf.data.Dataset.from_tensor_slices(training_images)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(training_labels)
test_features_dataset = tf.data.Dataset.from_tensor_slices(test_images)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(test_labels)

# 2. Combine features & labels
train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))


# 3. Batch and prefetch for optimal performance
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [29]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Rescaling(scale=1/255.), # Use a layer instead of the manual rescaling to take advantage of GPU and simulate a real testing environemnt where you can't access the testing dataset
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')
                                    ])

model.compile(optimizer = "adam",
              loss = 'sparse_categorical_crossentropy', # Sparse categorical because our multi class problem is not one hot encoded.
              metrics=['accuracy'])

model.fit(train_dataset, epochs=10, callbacks=callbacks, verbose=1)

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4320 - accuracy: 0.8441
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2901 - accuracy: 0.8945
Epoch 3/10
1868/1875 [============================>.] - ETA: 0s - loss: 0.2458 - accuracy: 0.9095
Reached 90% accuracy so cancelling training!
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2455 - accuracy: 0.9096


In [30]:
model.evaluate(test_dataset)

313/313 [==============================] - 1s 3ms/step - loss: 0.3083 - accuracy: 0.8891


[0.3083018660545349, 0.8891000151634216]